## Fake News Detection with Python

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import re
import string

Importing Data sets

In [2]:
fake_news = pd.read_csv('Fake.csv')
true_news = pd.read_csv('True.csv')

In [3]:
#fake_news data 
fake_news.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
#true_news data 
true_news.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
true_news.shape,fake_news.shape

((21417, 4), (23481, 4))

In [6]:
fake_news['class']=0
true_news['class']=1

In [7]:
# Removing data
for i in range(23470,23481,1):
    fake_news.drop([i],axis=0,inplace=True)

for i in range(23406,23417,1):
    fake_news.drop([i],axis=0,inplace=True)
    

In [10]:
merged_data = pd.concat([true_news,fake_news],axis=0)
merged_data.head()

,title,text,subject,date,class
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [11]:
required_df = merged_data.drop(["title","subject","date"],axis=1)
required_df.head()

,text,class
0,WASHINGTON (Reuters) - The head of a conservat...,1
1,WASHINGTON (Reuters) - Transgender people will...,1
2,WASHINGTON (Reuters) - The special counsel inv...,1
3,WASHINGTON (Reuters) - Trump campaign adviser ...,1
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,1


### Data Cleaning

In [12]:
def remove_un(data):
    data = data.lower()
    data = re.sub('\[.*?\]','',data)
    data = re.sub('\\W',' ',data)
    data = re.sub('https?://\S+|www.\S+','',data)
    data = re.sub('<.*?>+','',data)
    data = re.sub('[%s]'%re.escape(string.punctuation),'',data)
    data = re.sub('\n','',data)
    data = re.sub('\w*\d\w','',data)
    return data

In [13]:
required_df['text']= required_df['text'].apply(remove_un)

In [14]:
required_df.head()

,text,class
0,washington reuters the head of a conservat...,1
1,washington reuters transgender people will...,1
2,washington reuters the special counsel inv...,1
3,washington reuters trump campaign adviser ...,1
4,seattle washington reuters president donal...,1


### Vectorization

In [15]:
# dependent and independent variables for calculation
x = required_df['text']
y = required_df['class']

In [16]:
x_train, x_test, y_train,y_test= train_test_split(x,y,test_size=0.25)

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [18]:
vectorization = TfidfVectorizer()
xv_train  = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

### Logistic Regression

In [19]:
from sklearn.linear_model import LogisticRegression

In [20]:
LR = LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression()

In [21]:
LR.score(xv_test,y_test)

0.9872537659327926

In [22]:
pred_LR = LR.predict(xv_test)


In [23]:
print(classification_report(y_test,pred_LR))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      5858
           1       0.98      0.99      0.99      5361

    accuracy                           0.99     11219
   macro avg       0.99      0.99      0.99     11219
weighted avg       0.99      0.99      0.99     11219



### Decision Tree Classification

In [24]:
from sklearn.tree import DecisionTreeClassifier

In [25]:
DT  = DecisionTreeClassifier()
DT.fit(xv_train,y_train)

DecisionTreeClassifier()

In [26]:
DT.score(xv_test,y_test)

0.9957215438096086

In [27]:
pred_DT = DT.predict(xv_test)

In [28]:
print(classification_report(y_test,pred_DT))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5858
           1       1.00      1.00      1.00      5361

    accuracy                           1.00     11219
   macro avg       1.00      1.00      1.00     11219
weighted avg       1.00      1.00      1.00     11219



### Random Forest Classifier

In [29]:
from sklearn.ensemble import RandomForestClassifier

In [30]:
RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train,y_train)

RandomForestClassifier(random_state=0)

In [31]:
RFC.score(xv_test,y_test)

0.9913539531152509

In [32]:
pred_RFC = RFC.predict(xv_test)

In [33]:
print(classification_report(y_test,pred_RFC))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5858
           1       0.99      0.99      0.99      5361

    accuracy                           0.99     11219
   macro avg       0.99      0.99      0.99     11219
weighted avg       0.99      0.99      0.99     11219

